In [225]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from lxml import html
import scrapy
from time import sleep
import urllib3
import json
from selenium import webdriver
import random


In [226]:
def parser(link):

    encabezados = {
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36'
    }

    resp = requests.get(link, headers=encabezados, verify=False)
    resp = resp.text

    #soup = get_Soup('https://minciencias.gov.co/convocatorias/todas')
    parser = html.fromstring(resp)
    urllib3.disable_warnings()

    return parser

In [227]:
#Links Abiertos

def links_proyectos(parser):
    links_proyectos = []

    links_abiertos = parserT.xpath('//ul[@id="wrapper-instrumentos-open"]//a//@href')
    links_cerrados = parserT.xpath('//ul[@id="wrapper-instrumentos-close"]//a//@href')

    links_proyectos += links_abiertos 
    links_proyectos += links_cerrados

    estados = []
    for x in range(len(links_abiertos)):
        estados.append('Abierto')
    for x in range(len(links_cerrados)):
        estados.append('Cerrado')

        links = pd.DataFrame()
    links['Link_proyecto'] = links_proyectos
    links['Estado'] = estados

    return links

In [228]:
def titulo(parser_proyecto):

    titulo = parser_proyecto.xpath('//h3/text()')

    return titulo[0]

In [229]:
def fechas(parser_proyecto):
    """
    saca fechas de  apertura y cierre
    """
    fechas_general = parser_proyecto.xpath('//span[@class="date_text"]//text()')

    apertura = ''
    cierre = ''
    for i in range(len(fechas_general)):
        if fechas_general[i].lower() == 'apertura':
            apertura = fechas_general[i+1].strip(' - ')
        if fechas_general[i].lower() == 'cierre':
            cierre = fechas_general[i+1].strip(' - ')

    return apertura, cierre

In [230]:
def descripcion(parser_proyecto):
    
    descripcion_general = parser_proyecto.xpath('//div[@class="row_left"]//p/text()')

    descripcion = ''
    for i in descripcion_general:
        if len(i.strip()) != 0:
            descripcion = descripcion + i + ' '

    return descripcion.strip()

In [231]:
def pdfs(parser_proyecto):

    links_general = parser_proyecto.xpath('//div[@class="center_content text-left"]//a/@href')

    documentos_convocatoria = ''
    for link in links_general:
        if link.endswith('.pdf') or link.endswith('.docx'):
            documentos_convocatoria = documentos_convocatoria + link + ', '

    return documentos_convocatoria.strip(', ') 

In [232]:
def uruguay():
    parserT = parser('https://www.anii.org.uy/apoyos/investigacion/')
    links = links_proyectos(parserT)

    titulos = []
    fechas_apertura = []
    fechas_cierre = []
    descripciones = []
    pdf_proyecto = []

    for i in links['Link_proyecto']:
        parser_proyecto = parser(i)
        apertura, cierre = fechas(parser_proyecto)
        
        titulos.append(titulo(parser_proyecto))
        fechas_apertura.append(apertura)
        fechas_cierre.append(cierre)
        descripciones.append(descripcion(parser_proyecto))
        pdf_proyecto.append(pdfs(parser_proyecto))


    uruguay = pd.DataFrame()
    uruguay['Titulo'] = titulos
    uruguay['Fecha apertura'] = fechas_apertura
    uruguay['Fecha cierre'] = fechas_cierre
    uruguay['Descripción'] = descripciones
    uruguay['Pdfs'] = pdf_proyecto 
    uruguay['Link Proyecto'] = links['Link_proyecto']
    uruguay['Estado'] = links['Estado']

    uruguay['Titulo'] = uruguay['Titulo'].str.encode(encoding = 'latin1').str.decode(encoding = 'UTF-8', errors='ignore')
    uruguay['Descripción'] = uruguay['Descripción'].str.encode(encoding = 'latin1').str.decode(encoding = 'UTF-8', errors='ignore')
    
    return uruguay

In [233]:
uruguay = uruguay()
uruguay.to_excel('Uruguay.xlsx')